In [1]:
import pandas

import pygsheets
import numpy
import scipy
import re

import matplotlib
import matplotlib.pyplot as mp
import matplotlib.cm

import textwrap
import seaborn

In [2]:
pandas.__version__

'1.5.0'

# import data directly from google sheets

In [3]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
#spreadsheet = gc.open_by_key('1BshhQvcBOhqW4OPkMHXkZBdz9zYvS9q4ockW9yKufGQ') # specific doc for report
spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek') # original

#spreadsheet[1] "Gas Pipelines" tab is the second index
gas_pipes = spreadsheet.worksheet('title','Gas pipelines').get_as_df(start='A2')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A2')

pipes_df_orig = pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#get country ratios sheet
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()

In [4]:
owners_df_orig = spreadsheet.worksheet('title', 'Pipeline operators/owners (1/3)').get_as_df(start='A2')
owners_df_orig = owners_df_orig.loc[owners_df_orig.ProjectID!='']
owners_df_orig = owners_df_orig.loc[owners_df_orig.Wiki!='']
owners_df = owners_df_orig.replace('',numpy.nan)

owner_parent_links_df = spreadsheet.worksheet('title', 'Owner–parent relationships (2/3)').get_as_df(start='A2')
# only keep the owners with a checked relationship
owner_parent_links_df = owner_parent_links_df.loc[owner_parent_links_df['Parent–Owner Relationship Checked?']=='yes']
owner_parent_links_df.replace('',numpy.nan,inplace=True)

parents_df = spreadsheet.worksheet('title', 'Parent metadata (3/3)').get_as_df(start='A2')
parents_df = parents_df.loc[parents_df.Parent!='']

owners_df.set_index('ProjectID', inplace=True)
owner_parent_links_df.set_index('Owner', inplace=True)
parents_df.set_index('Parent', inplace=True)

# ****************************************

## create list of owner and parent column names
owner_pct_col_names = []
owner_col_names = []

parent_pct_col_names = []
parent_col_names = []

for num in range(1,11+1):
    owner_pct_col = f'Owner{num}%'
    owner_pct_col_names.append(owner_pct_col)
    
    owner_col = f'Owner{num}'
    owner_col_names.append(owner_col)
    
    parent_pct_col = f'Parent{num}%'
    parent_pct_col_names.append(parent_pct_col)
    
    parent_col = f'Parent{num}'
    parent_col_names.append(parent_col)

# ****************************************
## fill in missing parent info by borrowing owner info
owners_FULL_set = owners_df[owner_col_names].stack().dropna().unique().tolist() # from owners_df
owners_researched_set = list(set(owner_parent_links_df.index.to_list()))#+['Unknown'] # only existing owners, plus 'Unknown'
owners_diff = list(set(owners_FULL_set)-set(owners_researched_set))
owners_diff.append('Unknown')

# update owner_parent_links_df with these extra owners
owner_parent_links_df = pandas.concat([owner_parent_links_df, pandas.DataFrame(index=owners_diff, columns=owner_parent_links_df.columns)])
owner_parent_links_df['Parent1'].loc[owners_diff] = owners_diff
owner_parent_links_df['Parent1%'].loc[owners_diff] = '100.00%'

# ****************************************
# update parents_df with these as well
# note countries will be unknkown...
parents_set = list(set(parents_df.index.to_list()))
parents_diff = list(set(owners_diff)-set(parents_set))
parents_diff.append('Unknown')
parents_df = pandas.concat([parents_df, pandas.DataFrame(numpy.nan, index=parents_diff, columns=parents_df.columns)])
parents_df.loc[parents_diff,'ParentHQCountry'] = 'Unknown'

## replace "--" with NaN, removing empty rows

the dataset is structured to have -- wherever there's a lookup value that doesn't exist; replacing it with NaN (numpy.nan) allows pandas to treat it as a null value, which makes calculations much easier

In [5]:
# replace -- entries with NaN
pipes_df_orig = pipes_df_orig.replace('--', numpy.nan)
pipes_df_orig = pipes_df_orig[pipes_df_orig['PipelineName']!='']
pipes_df_orig = pipes_df_orig[pipes_df_orig['Wiki']!='']
country_ratios_df.replace('--', numpy.nan, inplace=True)

# km by country, km by region calculations

In [6]:
status_list = ['Proposed', 'Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']
country_list = sorted(list(set(country_ratios_df['Country'])))
region_list = sorted(list(set(country_ratios_df['Region'])))

In [7]:
country_ratios_gas_df = country_ratios_df[country_ratios_df['Fuel']=='Gas']

km_by_country_df = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region_df = pandas.DataFrame(columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_gas_df_status = country_ratios_gas_df[country_ratios_gas_df['Status']==status]
    km_by_country_df[status] = country_ratios_gas_df_status.groupby('Country')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_gas_df_status = country_ratios_gas_df[country_ratios_gas_df['Status']==status]
    km_by_region_df[status] = country_ratios_gas_df_status.groupby('Region')['MergedKmByCountry'].sum()

# # fill NaN with 0.0
km_by_region_df = km_by_region_df.fillna(0)
km_by_country_df = km_by_country_df.fillna(0)

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


In [8]:
km_by_country_df.index.to_list()

['Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Area of overlap Australia/Indonesia',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Benin',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 "Côte d'Ivoire",
 'DR Congo',
 'Denmark',
 'Disputed (Western Sahara/Mauritania)',
 'Djibouti',
 'Dominica',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Finland',
 'France',
 'Gabon',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guadeloupe',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Honduras',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Joint development area Australia/East

In [9]:
km_by_region_df['Proposed+Construction'] = km_by_region_df[['Proposed','Construction']].sum(axis=1)
km_by_region_df.sort_values(by='Proposed+Construction', inplace=True)
km_by_region_df = km_by_region_df[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
km_by_region_df.sort_values('Proposed+Construction', ascending=False, inplace=True)

In [10]:
km_by_country_df['Proposed+Construction'] = km_by_country_df[['Proposed','Construction']].sum(axis=1)
km_by_country_df.sort_values(by='Proposed+Construction', inplace=True)
km_by_country_df = km_by_country_df[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
km_by_country_df.sort_values('Proposed+Construction', ascending=False, inplace=True)

# projects by country, by region

In [11]:
country_ratios_gas_df = country_ratios_df[country_ratios_df['Fuel']=='Gas']

num_by_country_df = pandas.DataFrame(columns=status_list, index=country_list)
num_by_region_df = pandas.DataFrame(columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_gas_df_status = country_ratios_gas_df[country_ratios_gas_df['Status']==status]
    num_by_country_df[status] = country_ratios_gas_df_status.groupby('ProjectID')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_gas_df_status = country_ratios_gas_df[country_ratios_gas_df['Status']==status]
    num_by_region_df[status] = country_ratios_gas_df_status.groupby('Region')['MergedKmByCountry'].sum()

# # fill NaN with 0.0
num_by_region_df = num_by_region_df.fillna(0)
num_by_country_df = num_by_country_df.fillna(0)

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


# cost estimates (pipeline cost per km)

throwing out any estimates larger than USD 10 million per km for taking regional estimates

In [12]:
outliers_projectids = ['P2509','P0173','P1297', 'P4033', 'P0174','P3188','P2551', 'P4069', 'P0331', 'P1382','P0477']
# for gas...

In [13]:
# pull out only pipelines that have a KNOWN length AND a cost
country_ratios_with_length_and_cost_df = country_ratios_df.copy()[(country_ratios_df['Fuel']=='Gas') & 
                                    (country_ratios_df['CostUSDPerKm'].notna()) & 
                                    (country_ratios_df['LengthKnownKmByCountry'].notna()) &
                                    (country_ratios_df['LengthKnownKm']!=0) &
                                    #(country_ratios_df['CostUSDPerKm']<10e6) &
                                    ~(country_ratios_df['ProjectID'].isin(outliers_projectids))
                                   ]
#country_ratios_with_length_and_cost_df = country_ratios_with_length_and_cost_df[~country_ratios_with_length_and_cost_df.ProjectID.isin(outliers_projectids)]

### global mean value

In [14]:
country_ratios_with_length_and_cost_df['CostUSDPerKm'].drop_duplicates().mean()

3099032.3178019323

### calculate regional costs

In [15]:
pipes_costs_region_df = pandas.DataFrame(0, index=region_list, columns=['CostUSDPerKm','NumberOfCosts','NumberOfLengths'])

for region in region_list:
    print(region)
    country_ratios_region_df = country_ratios_with_length_and_cost_df[country_ratios_with_length_and_cost_df['Region']==region]
    pipes_costs_region_df.loc[region]['CostUSDPerKm'] = country_ratios_region_df['CostUSDPerKm'].mean()
    pipes_costs_region_df.loc[region]['NumberOfCosts'] = list(set(country_ratios_region_df['ProjectID'])).__len__()
    pipes_costs_region_df.loc[region]['NumberOfLengths'] = list(set(country_ratios_region_df['ProjectID'])).__len__()
    

Australia and New Zealand
East Asia
Eurasia
Europe
Latin America and the Caribbean
Middle East and North Africa
North America
SE Asia
South Asia
Sub-Saharan Africa


### table for methodology with regional cost info

In [16]:
pipes_costs_region_df

,CostUSDPerKm,NumberOfCosts,NumberOfLengths
Australia and New Zealand,0,10,10
East Asia,0,55,55
Eurasia,0,46,46
Europe,0,111,111
Latin America and the Caribbean,0,24,24
Middle East and North Africa,0,31,31
North America,0,140,140
SE Asia,0,8,8
South Asia,0,17,17
Sub-Saharan Africa,0,5,5


### calculate country-level costs

In [17]:
#country_list_for_costs = sorted(list(set(country_ratios_with_length_and_cost_df['Country'])))
pipes_costs_country_df = pandas.DataFrame(0, index=country_list, columns=['CostUSDPerKm','NumberOfCosts','NumberOfLengths'])

for country in country_list:#_for_costs:
    #print(country)
    country_ratios_country_df = country_ratios_with_length_and_cost_df[country_ratios_with_length_and_cost_df['Country']==country]
    pipes_costs_country_df.loc[country,'CostUSDPerKm'] = country_ratios_country_df['CostUSDPerKm'].mean()
    pipes_costs_country_df.loc[country,'NumberOfCosts'] = list(set(country_ratios_country_df['ProjectID'])).__len__()
    pipes_costs_country_df.loc[country,'NumberOfLengths'] = list(set(country_ratios_country_df['ProjectID'])).__len__()

# tables etc.

## table for regional totals

In [18]:
if 'Total' in km_by_region_df.index:
    km_by_region_df.drop(index='Total', inplace=True)   
    total = km_by_region_df.sum(axis=0)
    total.name='Total'
    km_by_region_df = km_by_region_df.append(total)

else:
    total = km_by_region_df.sum(axis=0)
    total.name='Total'
    km_by_region_df = km_by_region_df.append(total)

km_by_region_df

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_99585/997741405.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  km_by_region_df = km_by_region_df.append(total)


,Proposed,Construction,Proposed+Construction,Shelved,Cancelled,Operating,Idle,Mothballed,Retired
East Asia,31380.90,26377.70,57758.60,2065.71,475.27,92288.09,0.0,0.00,41.00
Eurasia,18321.40,5153.38,23474.78,272.16,6647.69,128934.76,0.0,3300.05,1784.94
South Asia,4221.03,17140.20,21361.23,6357.71,6143.81,29669.32,0.0,0.00,35.00
Sub-Saharan Africa,18823.08,1582.00,20405.08,1678.15,176.96,7031.52,0.0,0.00,0.00
Europe,14399.21,2836.20,17235.41,5705.03,14014.71,110152.27,0.0,4734.21,821.36
North America,11837.51,2768.33,14605.84,10916.42,13243.21,420872.56,331.0,0.00,0.00
Latin America and the Caribbean,12772.26,1304.00,14076.26,2187.28,1301.38,39241.07,0.0,224.00,0.00
Australia and New Zealand,12563.28,580.00,13143.28,488.23,6194.69,31025.88,0.0,0.00,0.00
Middle East and North Africa,5269.15,6726.91,11996.06,5963.24,4144.74,44502.38,0.0,1106.60,39.28
SE Asia,9222.66,984.30,10206.96,2634.21,1423.81,16879.66,0.0,0.00,0.00


## table for km by country

In [19]:
# saves all countries to an Excel file for reference

km_by_country_df.sort_values('Proposed+Construction', ascending=False)

if 'Total' in km_by_country_df.index:
    km_by_country_df.drop(index='Total', inplace=True)   
    total = km_by_country_df.sum(axis=0)
    total.name='Total'
    km_by_country_df = km_by_country_df.append(total)

else:
    total = km_by_country_df.sum(axis=0)
    total.name='Total'
    km_by_country_df = km_by_country_df.append(total)

#km_by_country_df.to_excel('GGIT-km-by-country.xlsx')

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_99585/464806905.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  km_by_country_df = km_by_country_df.append(total)


# table for stranded asset calculations

## country-level capex estimates

In [20]:
country_ratios_gas_df

,PipelineName,SegmentName,ProjectID,Country,LengthEstimateKmByCountry,LengthEstimateKm,LengthPerCountryFraction,Owner,StartYearEarliest,Region,...,CostUSD,CostEuro,LengthKnownKm,LengthKnownKmByCountry,MergedKmByPipeline,CostUSDPerKm,CostEuroPerKm,MergedKmByCountry,Owner,Parent
0,Turkmenistan-Afghanistan-Pakistan-India Gas Pi...,,P0766,Afghanistan,1033.682603,1814.0,0.50,Turkmengaz (100%),NaN,Middle East and North Africa,...,10000000000.0,8.771930e+09,1814.0,907.05,1814.0,5512679.16,4835683.48,907.05,Turkmengaz [100.00%],Turkmengaz [100.00%]
2,Sheberghan-Mazar-i-Sharif Gas Pipeline,,P3922,Afghanistan,129.650156,94.5,1.00,NaN,NaN,Middle East and North Africa,...,NaN,NaN,94.5,94.50,94.5,NaN,NaN,94.50,Unknown [unknown %],Unknown [unknown %]
4,Ionian Adriatic Gas Pipeline,,P0702,Albania,165.029149,540.0,0.33,Albgaz Sha; BH-Gas d.o.o.; Plinacro; Montenegr...,2025,Europe,...,668040000.0,5.860000e+08,540.0,178.31,540.0,1237111.11,1085185.19,178.31,Albgaz Sha [unknown %]; BH-Gas d.o.o. [unknown...,Albgaz Sha [unknown %]; BH-Gas d.o.o. [unknown...
5,Trans-Adriatic Gas Pipeline,,P0724,Albania,215.354312,878.0,0.29,Trans Adriatic Pipeline AG (100%),2020,Europe,...,4500000000.0,3.947368e+09,878.0,251.27,878.0,5125284.74,4495863.81,251.27,Trans Adriatic Pipeline AG [600.00%],BP [20.00%]; SOCAR [20.00%]; Snam [20.00%]; Fl...
6,South Stream Gas Pipeline,,P0760,Albania,18.313699,2380.0,0.01,South Stream B.V.,NaN,Europe,...,15500000000.0,1.359649e+10,2380.0,12.03,2380.0,6512605.04,5712811.44,12.03,South Stream B.V. [unknown %],Agence des participations de l'État [unknown %...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4377,FSRU to Thai Binh Gas Pipeline,,P3151,Vietnam,50,50.0,1.00,NaN,2026,SE Asia,...,NaN,NaN,50.0,50.00,50.0,NaN,NaN,50.00,Unknown [unknown %],Unknown [unknown %]
4378,East to Southwest Gas Pipeline,,P3152,Vietnam,,,1.00,NaN,2026,SE Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown [unknown %],Unknown [unknown %]
4379,Trans-ASEAN Gas Pipeline,,P3153,Vietnam,,,1.00,NaN,2026,SE Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown [unknown %],Unknown [unknown %]
4380,Puerto Rico-Virgin Islands Pipeline,,P1253,Virgin Islands (U.S.),27.247023,,0.66,NaN,NaN,Latin America and the Caribbean,...,NaN,NaN,NaN,NaN,41.44,NaN,NaN,27.25,Unknown [unknown %],Unknown [unknown %]


In [21]:
country_ratios_gas_df = country_ratios_df.loc[country_ratios_df['Fuel']=='Gas']
country_ratios_gas_df = country_ratios_gas_df.reset_index(drop=True)
country_ratios_gas_df.loc[:,'CostUSDEstimate'] = numpy.nan

for idx,row in country_ratios_gas_df.iterrows():
    # calculate cost
    cntry = row.Country
    region = row.Region
    km_by_cntry = row.LengthKnownKmByCountry
    #print(row.PipelineName)
    country_ratios_gas_df.loc[idx,'CostUSDEstimate'] = pipes_costs_region_df.loc[region, 'CostUSDPerKm'] * km_by_cntry

# replace any known costs now

# known_costs = country_ratios_gas_df.loc[(~country_ratios_gas_df.LengthKnownKmByCountry.isna())&
#                       (~country_ratios_gas_df.CostUSDPerKm.isna()),'LengthKnownKmByCountry'] * \
# country_ratios_gas_df.loc[(~country_ratios_gas_df.LengthKnownKmByCountry.isna())&
#                       (~country_ratios_gas_df.CostUSDPerKm.isna()),'CostUSDPerKm']

# country_ratios_gas_df.loc[(~country_ratios_gas_df.LengthKnownKmByCountry.isna())&
#                       (~country_ratios_gas_df.CostUSDPerKm.isna()),'CostUSDEstimate'] = known_costs

country_ratios_gas_df.loc[(~country_ratios_gas_df.LengthKnownKmByCountry.isna())&
                      (~country_ratios_gas_df.CostUSDPerKm.isna()),'CostUSDEstimate'] = \
country_ratios_gas_df.loc[(~country_ratios_gas_df.LengthKnownKmByCountry.isna())&
                      (~country_ratios_gas_df.CostUSDPerKm.isna()),'LengthKnownKmByCountry'] * \
country_ratios_gas_df.loc[(~country_ratios_gas_df.LengthKnownKmByCountry.isna())&
                      (~country_ratios_gas_df.CostUSDPerKm.isna()),'CostUSDPerKm']

country_sums_USDBillion_df = pandas.DataFrame(country_ratios_gas_df.loc[(country_ratios_gas_df.Status.isin(['Construction','Proposed']))].groupby('Country')['CostUSDEstimate'].sum(min_count=1)/1e9)

In [22]:
capex_by_country_df = pandas.DataFrame(columns=status_list, index=country_list)
capex_by_region_df = pandas.DataFrame(columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_gas_df_status = country_ratios_gas_df[country_ratios_gas_df.Status==status]
    capex_by_country_df[status] = country_ratios_gas_df_status.groupby('Country')['CostUSDEstimate'].sum()/1e9

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_gas_df_status = country_ratios_gas_df[country_ratios_gas_df.Status==status]
    capex_by_region_df[status] = country_ratios_gas_df_status.groupby('Region')['CostUSDEstimate'].sum()/1e9

# # fill NaN with 0.0
capex_by_region_df = capex_by_region_df.fillna(0)
capex_by_country_df = capex_by_country_df.fillna(0)

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


In [23]:
capex_by_region_df['Proposed+Construction'] = capex_by_region_df[['Proposed','Construction']].sum(axis=1)
#capex_by_region_df.sort_values(by='Proposed+Construction', inplace=True)
capex_by_region_df = capex_by_region_df[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
capex_by_region_df.sort_values('Proposed+Construction', ascending=False, inplace=True)

In [24]:
capex_by_country_df['Proposed+Construction'] = capex_by_country_df[['Proposed','Construction']].sum(axis=1)
#capex_by_country_df.sort_values(by='Proposed+Construction', inplace=True)
capex_by_country_df = capex_by_country_df[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
capex_by_country_df.sort_values('Proposed+Construction', ascending=False, inplace=True)

In [25]:
capex_by_region_df

,Proposed,Construction,Proposed+Construction,Shelved,Cancelled,Operating,Idle,Mothballed,Retired
Sub-Saharan Africa,32.097253,2.890000,34.987253,0.000000,0.000000,0.000000,0.0,0.0,0.0
Europe,30.217674,4.346361,34.564035,14.508965,13.357027,57.978817,0.0,0.0,0.0
North America,26.557855,7.155367,33.713223,20.319857,65.320740,50.995779,0.0,0.0,0.0
Middle East and North Africa,18.074621,13.650526,31.725148,0.596237,2.684083,25.219216,0.0,0.0,0.0
Latin America and the Caribbean,18.057196,0.051528,18.108724,6.658980,0.000000,0.000000,0.0,0.0,0.0
Eurasia,13.234595,1.148518,14.383114,1.072786,6.722900,87.696398,0.0,0.0,0.0
South Asia,0.834167,11.404557,12.238724,0.817500,5.419998,1.348014,0.0,0.0,0.0
East Asia,5.319495,2.651321,7.970816,0.041760,2.122873,5.966114,0.0,0.0,0.0
Australia and New Zealand,7.200839,0.000000,7.200839,0.000000,0.094428,0.685068,0.0,0.0,0.0
SE Asia,3.163400,1.300000,4.463400,0.312313,0.000000,0.000000,0.0,0.0,0.0


In [26]:
capex_by_region_df

,Proposed,Construction,Proposed+Construction,Shelved,Cancelled,Operating,Idle,Mothballed,Retired
Sub-Saharan Africa,32.097253,2.890000,34.987253,0.000000,0.000000,0.000000,0.0,0.0,0.0
Europe,30.217674,4.346361,34.564035,14.508965,13.357027,57.978817,0.0,0.0,0.0
North America,26.557855,7.155367,33.713223,20.319857,65.320740,50.995779,0.0,0.0,0.0
Middle East and North Africa,18.074621,13.650526,31.725148,0.596237,2.684083,25.219216,0.0,0.0,0.0
Latin America and the Caribbean,18.057196,0.051528,18.108724,6.658980,0.000000,0.000000,0.0,0.0,0.0
Eurasia,13.234595,1.148518,14.383114,1.072786,6.722900,87.696398,0.0,0.0,0.0
South Asia,0.834167,11.404557,12.238724,0.817500,5.419998,1.348014,0.0,0.0,0.0
East Asia,5.319495,2.651321,7.970816,0.041760,2.122873,5.966114,0.0,0.0,0.0
Australia and New Zealand,7.200839,0.000000,7.200839,0.000000,0.094428,0.685068,0.0,0.0,0.0
SE Asia,3.163400,1.300000,4.463400,0.312313,0.000000,0.000000,0.0,0.0,0.0


In [27]:
total

Proposed                 138810.48
Construction              65453.02
Proposed+Construction    204263.50
Shelved                   38268.14
Cancelled                 53766.27
Operating                920597.51
Idle                        331.00
Mothballed                 9364.86
Retired                    2721.58
Name: Total, dtype: float64

In [28]:
if 'Total' in capex_by_region_df.index:
    capex_by_region_df.drop(index='Total', inplace=True)   
    total = capex_by_region_df.sum(axis=0)
    #total.name='Total'
    capex_by_region_df.loc['Total'] = total

else:
    total = capex_by_region_df.sum(axis=0)
    #total.name='Total'
    capex_by_region_df.loc['Total'] = total

In [29]:
if 'Total' in capex_by_country_df.index:
    capex_by_country_df.drop(index='Total', inplace=True)   
    total = capex_by_country_df.sum(axis=0)
    #total.name='Total'
    capex_by_country_df.loc['Total'] = total

else:
    total = capex_by_country_df.sum(axis=0)
    #total.name='Total'
    capex_by_country_df.loc['Total'] = total

## print out country-level stranded assets for Jenny's report

In [32]:
iom = pandas.Series(numpy.zeros(capex_by_country_df.columns.__len__()))#,dtype='float64')
iom.index = capex_by_country_df.columns
iom.name='Macao'
if 'Macao' not in capex_by_country_df.index:
    capex_by_country_df = capex_by_country_df.append(iom)

if 'Macao' not in km_by_country_df.index:
    km_by_country_df = km_by_country_df.append(iom)

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_99585/4198377808.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  capex_by_country_df = capex_by_country_df.append(iom)
/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_99585/4198377808.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  km_by_country_df = km_by_country_df.append(iom)


In [40]:
country_list_gas_plants_report = [ \
"Algeria",
"Angola",
"Benin",
"Botswana",
"Burkina Faso",
"Burundi",
"Cabo Verde",
"Cameroon",
"Central African Republic",
"Chad",
"Comoros",
"Congo",
"Côte d'Ivoire",
"Djibouti",
"DR Congo",
"Egypt",
"Equatorial Guinea",
"Eritrea",
"Eswatini",
"Ethiopia",
"Gabon",
"Gambia",
"Ghana",
"Guinea",
"Guinea-Bissau",
"Kenya",
"Lesotho",
"Liberia",
"Libya",
"Madagascar",
"Malawi",
"Mali",
"Mauritania",
"Mauritius",
"Morocco",
"Mozambique",
"Namibia",
"Niger",
"Nigeria",
"Rwanda",
"Sao Tome and Principe",
"Senegal",
"Seychelles",
"Sierra Leone",
"Somalia",
"South Africa",
"South Sudan",
"Sudan",
"Tanzania",
"Togo",
"Tunisia",
"Uganda",
"Zambia",
"Zimbabwe" ]

pandas.options.display.float_format = '{:,.5f}'.format
capex_by_country_df.loc[capex_by_country_df.index.isin(country_list_gas_plants_report),['Proposed','Construction']].reindex(country_list_gas_plants_report)

,Proposed,Construction
Algeria,4.07198,0.00000
Angola,0.00000,0.00000
Benin,0.76162,0.00000
Botswana,NaN,NaN
Burkina Faso,NaN,NaN
Burundi,NaN,NaN
Cabo Verde,NaN,NaN
Cameroon,0.00000,0.00000
Central African Republic,0.00000,0.00000
Chad,0.00000,0.00000


In [42]:
km_by_country_df.loc[km_by_country_df.index.isin(country_list_gas_plants_report),['Proposed','Construction','Operating']].reindex(country_list_gas_plants_report)

,Proposed,Construction,Operating
Algeria,"1,293.01000",0.00000,"13,627.92000"
Angola,0.00000,0.00000,384.68000
Benin,172.43000,0.00000,119.71000
Botswana,NaN,NaN,NaN
Burkina Faso,NaN,NaN,NaN
Burundi,NaN,NaN,NaN
Cabo Verde,NaN,NaN,NaN
Cameroon,0.00000,0.00000,0.00000
Central African Republic,0.00000,0.00000,0.00000
Chad,0.00000,0.00000,0.00000
